In [1]:
%matplotlib inline
import os, sys, numpy as np, pandas as pd, tensorflow as tf, re, codecs, seaborn as sns, json, time, csv, datetime as dt
import pickle, collections, random, math, numbers, scipy.sparse as sp, matplotlib.pyplot as plt, scipy.sparse as sp

from pprint import pprint
from tensorflow.contrib.training.python.training.hparam import HParams

def reload(mName):
    import importlib
    if mName in sys.modules:
        del sys.modules[mName]
    return importlib.import_module(mName)


from collections import deque, defaultdict, OrderedDict
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, minmax_scale
from matplotlib import pyplot as plt
plt.style.use('ggplot')

# classpath
# ctx = os.path.abspath('..').replace('\\', '/')
ctx = 'D:/Python/notebook/restful'
cps = [ctx]
_ = [sys.path.insert(0, cp) for cp in cps if cp not in sys.path]

# data path
datapath = '/'.join([ctx, 'data'])

seed = 88
utils = reload('recomm.trainer.utils.utils')
np.set_printoptions(precision=4, suppress=True, linewidth=100)
np.random.seed(seed)

D:\Python\Anaconda3\envs\py3_5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Simple Data Preprocess

In [55]:
import datetime as dt

ratings = pd.read_csv("{}/ml-latest-small/ratings.csv".format(datapath))
ratings['timestamp'] = ratings.timestamp.map(dt.datetime.fromtimestamp).map(str)
ratings['ori_rating'] = ratings['rating']
ratings['rating'] = (ratings.rating >= 4).astype(int)
tr, te = utils.split_by_ratio(ratings)

movies = pd.read_csv("{}/ml-latest-small/movies.csv".format(datapath))
avg_rt = ratings.groupby("movieId", as_index=False).ori_rating.mean().rename(index=str, columns={'ori_rating': 'avg_rating'})
movies = movies.merge(avg_rt, how='left', on='movieId')
# movies.avg_rating.fillna(ratings.rating.mean())
movies["year"] = movies.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)
# movies["year"] = minmax_scale(movies.year.fillna(movies.year.median()))

In [ ]:
def preprocess(data, movie_trans, train_hist=None, is_train=True):
    queue = []
    data = data.merge(movie_trans, how="left", on="movieId")
    columns=["user_id", "query_movie_ids",
             "genres", "avg_rating", "year", "candidate_movie_id",
             "timestamp",
             "rating"]
    
    list2str = lambda lst: ','.join(map(str, lst))
    for u, df in data.groupby("userId"):
        df = df.sort_values("rating", ascending=False)
        if not is_train:
            user_movies_hist = train_hist.query("userId == {}".format(u)).movieId
        for i, (_, r) in enumerate(df.iterrows()):
            if is_train:
                query_hist = df.movieId[:i].tolist() + df.movieId[i + 1:].tolist()
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
            else:
                tr_hist = set(user_movies_hist.tolist())
                query_hist = list(tr_hist - set([int(r.movieId)]))
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
    return pd.DataFrame(queue, columns=columns)
    
tr_merged = preprocess(tr, movies)
tr_merged.to_csv('./tr.raw.movielens.csv', index=False, header=None)

te_merged = preprocess(te, movies, tr, is_train=False)
te_merged.to_csv('./te.raw.movielens.csv', index=False, header=None)
# 合併成一個檔案
merged = pd.concat([tr_merged, te_merged], ignore_index=True)
merged.to_csv('./merged_movielens.csv', index=False, header=None)
merged.head()

<br/>
<br/>
<br/>
## Cmd Submit Training

In [ ]:
!cd D:/Python/notebook/recomm_prod && \
gcloud ml-engine jobs submit training recomm_movielens_15 \
    --job-dir gs://recomm-job/foo/model \
    --runtime-version 1.4 \
    --module-name trainer.ctrl \
    --package-path trainer \
    --region asia-east1 \
    --config config.yaml \
    -- \
    --method train \
    --conf-path gs://recomm-job/foo/data/user_supplied/movielens.yaml

In [ ]:
!gcloud ml-engine jobs describe recomm_movielens_15

## Restful API Test

In [69]:
import requests

resp = requests.get('http://127.0.0.1:8000/api/recomm/load_schema', 
                    params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'})
print( resp.text )

<!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta name="robots" content="NONE,NOARCHIVE">
  <title>TypeError
          at /api/recomm/load_schema/</title>
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background-color:#fff; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; }
    h2 { margin-bottom:.8em; }
    h2 span { font-size:80%; color:#666; font-weight:normal; }
    h3 { margin:1em 0 .5em 0; }
    h4 { margin:0 0 .5em 0; font-weight: normal; }
    code, pre { font-size: 100%; white-space: pre-wrap; }
    table { border:1px solid #ccc; border-collapse: collapse; width:100%; background:white; }
    tbody td, tbody th { vertical-align:top; padding:2px 3px; }
    thead th {
      padding:1px 6px 1px 3px; background:#fefefe; text-align:left;
      font-weight:normal; fo

<br/>
<br/>
<br/>
## Python Client API Transform Data

In [ ]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
hparam = dict(conf_path='gs://movielens-foo/user_supplied/movielens.yaml')
views.gen_data(hparam)

## View Schema

In [4]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
loader = views.load_schema(params)

vars(loader.schema)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

<br/>
<br/>
<br/>
## Python Client API Submit Training

In [4]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml',
          'runtime_version': '1.4'}
ret = views.train_submit(params)
job_id = ret.get('job_id')
print( ret.get('response') )

ImportError: No module named 'recomm'

<br/>
<br/>
<br/>
## Describe Job States

In [155]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml', 'job_id': job_id}
ret = views.describe(params)
ret.get('response')

2018-03-08 15:22:28,470 - googleapiclient.discovery - INFO [line:274] - URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest
2018-03-08 15:22:29,763 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-engine/jobs/foo_bar_movielens_recommendation_20180308152221665827?alt=json
2018-03-08 15:22:29,765 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-08 15:22:29,798 - oauth2client.client - INFO [line:795] - Refreshing access_token
2018-03-08 15:22:31,101 - Ctrl - INFO [line:157] - foo-bar: describe take time 0:00:02.643986


{'createTime': '2018-03-08T07:22:26Z',
 'jobId': 'foo_bar_movielens_recommendation_20180308152221665827',
 'state': 'PREPARING',
 'trainingInput': {'args': ['--train-steps',
   '1000',
   '--method',
   'train',
   '--conf-path',
   'gs://movielens-foo/user_supplied/movielens.yaml',
   '--job-id',
   'foo_bar_movielens_recommendation_20180308152221665827'],
  'jobDir': 'gs://recomm-job/foo-bar/movielens_recommendation/model',
  'packageUris': ['gs://recomm-job/foo-bar/movielens_recommendation/model/packages/6a7f7097e2b3e0e19f97e673596db54eaeeeebba2e5e65a7c1bd959a20ed6dc7/trainer-0.1.tar.gz'],
  'pythonModule': 'trainer.ctrl',
  'pythonVersion': '3.5',
  'region': 'asia-east1',
  'runtimeVersion': '1.4'},
 'trainingOutput': {}}

## Deploy

In [232]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
ret = views.deploy(params)
ret

2018-03-09 15:56:54,409 - Service - INFO [line:97] - try to create model [foo_bar_movielens_recommendation] ...
2018-03-09 15:56:54,417 - googleapiclient.discovery - INFO [line:868] - URL being requested: POST https://ml.googleapis.com/v1/projects/training-recommendation-engine/models?alt=json
2018-03-09 15:56:54,419 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-09 15:56:54,451 - oauth2client.client - INFO [line:795] - Refreshing access_token
2018-03-09 15:56:55,773 - Service - WARNING [line:134] - <HttpError 409 when requesting https://ml.googleapis.com/v1/projects/training-recommendation-engine/models?alt=json returned "Field: model.name Error: A model with the same name already exists.">
2018-03-09 15:56:55,774 - Service - INFO [line:100] - try to clean old version ...
2018-03-09 15:56:55,781 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-e

{'err_cde': '00',
 'response': {'metadata': {'@type': 'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
   'createTime': '2018-03-09T07:56:58Z',
   'modelName': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation',
   'operationType': 'CREATE_VERSION',
   'version': {'createTime': '2018-03-09T07:56:56Z',
    'deploymentUri': 'gs://recomm-job/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839',
    'description': '[foo-bar] recommendation model',
    'name': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180309155657058554',
    'runtimeVersion': '1.4'}},
  'name': 'projects/training-recommendation-engine/operations/create_foo_bar_movielens_recommendation_v20180309155657058554-1520582216647'}}

## Get Information From Deployed Model

In [235]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
ret = views.model_info(params)
ret

2018-03-09 15:58:44,589 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180309155657058554?alt=json
2018-03-09 15:58:44,591 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-09 15:58:44,628 - oauth2client.client - INFO [line:795] - Refreshing access_token


{'err_cde': '00',
 'response': {'createTime': '2018-03-09T07:56:56Z',
  'deploymentUri': 'gs://recomm-job/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839',
  'description': '[foo-bar] recommendation model',
  'isDefault': True,
  'name': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180309155657058554',
  'runtimeVersion': '1.4',
  'state': 'READY'}}

## Restful predict

In [3]:
ratings = pd.read_csv("{}/ml-latest-small/ratings.csv".format(datapath))
ratings['timestamp'] = ratings.timestamp.map(dt.datetime.fromtimestamp).map(str)
ratings['ori_rating'] = ratings['rating']
ratings['rating'] = (ratings.rating >= 4).astype(int)

movies = pd.read_csv("{}/ml-latest-small/movies.csv".format(datapath))
avg_rt = ratings.groupby("movieId", as_index=False).ori_rating.mean().rename(index=str, columns={'ori_rating': 'avg_rating'})
movies = movies.merge(avg_rt, how='left', on='movieId')
movies["year"] = movies.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.yaml'}
loader = ctrl.load_schema(params)
merged = pd.read_csv('../data/foo/user_supplied/raws/merged_movielens.csv', names=loader.schema.raw_cols)
merged.head()

2018-03-09 09:21:22,462 - Loader - INFO [line:363] - try to unserialize from gs://recomm-job/foo-bar/movielens_recommendation/data/parsed.yaml


,user_id,query_movie_ids,genres,avg_rating,year,candidate_movie_id,timestamp,rating
0,1,"1953,2105,31,1029,1061,1129,1263,1287,1293,133...",Drama,4.260870,1989.0,1172,2009-12-14 10:53:25,1
1,1,"1172,2105,31,1029,1061,1129,1263,1287,1293,133...",Action|Crime|Thriller,4.021739,1971.0,1953,2009-12-14 10:53:11,1
2,1,"1172,1953,31,1029,1061,1129,1263,1287,1293,133...",Action|Adventure|Sci-Fi,3.478723,1982.0,2105,2009-12-14 10:52:19,1
3,1,"1172,1953,2105,1029,1061,1129,1263,1287,1293,1...",Drama,3.178571,1995.0,31,2009-12-14 10:52:24,0
4,1,"1172,1953,2105,31,1061,1129,1263,1287,1293,133...",Animation|Children|Drama|Musical,3.702381,1941.0,1029,2009-12-14 10:52:59,0


In [5]:
def restful_data(user_ids, num=5):
    data = {
        'query_movie_ids': merged.query('user_id in {}'.format(user_ids)).groupby('user_id').query_movie_ids.max().tolist(),
    }
    items = movies.rename(index=str, columns={"movieId": "candidate_movie_id"}).drop('title', 1)
    items.loc[:, 'candidate_movie_id'] = items.candidate_movie_id.astype(str)
    # reduce to 5 records
    items = items[:num].to_dict('list')
    data.update(items)
    return data
    
restful_data((22,))

{'avg_rating': [3.8724696356275303,
  3.4018691588785046,
  3.1610169491525424,
  2.3846153846153846,
  3.267857142857143],
 'candidate_movie_id': ['1', '2', '3', '4', '5'],
 'genres': ['Adventure|Animation|Children|Comedy|Fantasy',
  'Adventure|Children|Fantasy',
  'Comedy|Romance',
  'Comedy|Drama|Romance',
  'Comedy'],
 'query_movie_ids': ['32,1884,1580,1527,1387,1377,1376,1375,1372,1356,1339,1291,1270,1255,1240,1215,1210,1200,2174,2288,2291,3081,4011,3868,3793,3535,3527,3355,3213,3033,2459,2987,2959,2858,2762,2712,2571,2542,1198,1799,457,648,555,480,551,589,541,592,593,253,260,163,296,858,1089,2881,235,2990,2953,231,3052,208,2985,1148,2763,3147,267,2723,2717,2716,2710,2701,2700,2683,2672,3082,3300,3176,3751,4015,47,3999,3996,3994,3977,48,3826,3809,70,3697,173,3623,3578,153,158,3438,3408,3354,2617,3285,3253,2657,356,2616,588,1693,1682,1645,1641,1625,1608,1544,1391,552,586,1374,2605,1371,1320,784,1263,785,1080,1214,1208,1201,1097,1721,1722,1769,44,315,355,2502,1101,2431,2402,2340,230

In [236]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml',
          'json_data': restful_data((22,))}
ret = views.predict(params)
ret.get('response')

2018-03-09 15:58:52,172 - Loader - INFO [line:363] - try to unserialize from gs://recomm-job/foo-bar/movielens_recommendation/data/parsed.yaml
2018-03-09 15:58:53,879 - Loader - INFO [line:451] - try to restful transform ... 
2018-03-09 15:58:54,044 - googleapiclient.discovery - INFO [line:868] - URL being requested: POST https://ml.googleapis.com/v1/projects/training-recommendation-engine/models/foo_bar_movielens_recommendation:predict?alt=json
2018-03-09 15:58:54,045 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-09 15:58:54,079 - oauth2client.client - INFO [line:795] - Refreshing access_token
2018-03-09 15:58:55,714 - Ctrl - INFO [line:355] - foo-bar: predict take time 0:00:03.737939


{'predictions': [{'emb_item': [0.6766782402992249,
    -0.012259799987077713,
    0.33944520354270935,
    0.20576968789100647,
    -0.11439434438943863,
    0.030402852222323418,
    0.1481526643037796,
    0.14686912298202515,
    -0.46256718039512634,
    -0.19473426043987274,
    -0.05570128187537193,
    -0.05927087739109993,
    -0.2609875798225403,
    0.08330384641885757,
    -0.15491582453250885,
    0.042215242981910706,
    0.4620720148086548,
    -0.3400868773460388,
    0.0060962652787566185,
    -0.3157203793525696,
    0.0727677121758461,
    0.09835103154182434,
    0.026547472923994064,
    -0.45345425605773926,
    0.19915218651294708,
    0.2104223370552063,
    0.42013517022132874,
    -0.09830223023891449,
    -0.24704451858997345,
    0.36613404750823975,
    0.05687137693166733,
    0.2679730951786041],
   'emb_query': [-0.5186209082603455,
    -0.014119209721684456,
    -0.4282258450984955,
    -0.292388379573822,
    0.19547195732593536,
    -0.0142602575942873

## Local Run Saved Model

In [252]:
from tensorflow.contrib import predictor

utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
reload('recomm.trainer.reco_mf_dnn_est')
service = reload('recomm.trainer.service')

data = restful_data((22,))
views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 'is_local': True, 'json_data': data}
data_for_model = views.transform(params).get('response')

del data_for_model['query_movie_ids']
pred_fn = predictor.from_saved_model('../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839')
pred_fn(data_for_model)

2018-03-09 18:01:21,598 - Loader - INFO [line:363] - try to unserialize from D:\Python\notebook\recomm_prod\repo/foo-bar/movielens_recommendation/data/parsed.yaml
2018-03-09 18:01:23,632 - Loader - INFO [line:451] - try to restful transform ... 
2018-03-09 18:01:23,649 - Ctrl - INFO [line:335] - foo-bar: transform take time 0:00:02.051455
INFO:tensorflow:Restoring parameters from b'../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839\\variables\\variables'
2018-03-09 18:01:24,142 - tensorflow - INFO [line:116] - Restoring parameters from b'../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839\\variables\\variables'


InvalidArgumentError: You must feed a value for placeholder tensor 'query_movie_ids' with dtype int32 and shape [?,?]
	 [[Node: query_movie_ids = Placeholder[dtype=DT_INT32, shape=[?,?], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'query_movie_ids', defined at:
  File "D:\Python\Anaconda3\envs\py3_5\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Python\Anaconda3\envs\py3_5\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Python\Anaconda3\envs\py3_5\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Python\Anaconda3\envs\py3_5\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Python\Anaconda3\envs\py3_5\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Python\Anaconda3\envs\py3_5\lib\site-packages\zmq\eventloop\ioloop.py", li

## Inspect Saved Model

In [249]:
with tf.Session() as sess:
    tf.saved_model.loader.load(sess, 
        [tf.saved_model.tag_constants.SERVING],
        export_dir='../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839')
    for n in sess.graph.as_graph_def().node:
        print(n.name)

INFO:tensorflow:Restoring parameters from b'../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839\\variables\\variables'
2018-03-09 17:33:34,164 - tensorflow - INFO [line:116] - Restoring parameters from b'../repo/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520581839\\variables\\variables'
filenames
compression_type
buffer_size
num_parallel_calls
num_parallel_calls_1
buffer_size_1
seed
seed2
count
batch_size
Const
Const_1
Const_2
Const_3
Const_4
Const_5
Const_6
Const_7
padding_value
padding_value_1
padding_value_2
padding_value_3
padding_value_4
padding_value_5
padding_value_6
padding_value_7
OneShotIterator
IteratorToStringHandle
IteratorGetNext
init
global_step/Initializer/zeros/shape_as_tensor
global_step/Initializer/zeros/Const
global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
query_movie_ids
genres
avg_rating
year
candidate_movie_id
query_movie_ids_len
genres_len
init/random_uniform/shape
init/random_uniform/m

## Dataset Test

In [207]:
# model = result.get('response')
# vars(model)
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 
          'is_local': True,
          'runtime_version': '1.4',
          'train_steps': 600}

tf.reset_default_graph()
model, p = views.get_model(params)
train_input = model.input_fn2([p.train_file], n_epoch=1, n_batch=5)
features, labels = train_input()
print('\nfeatures: ', features)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print( sess.run(features) )

2. feat OrderedDict([('query_movie_ids', <tf.Tensor 'arg3:0' shape=(?,) dtype=int32>), ('genres', <tf.Tensor 'arg2:0' shape=(?,) dtype=int32>), ('avg_rating', <tf.Tensor 'arg0:0' shape=(1,) dtype=float32>), ('year', <tf.Tensor 'arg5:0' shape=(1,) dtype=float32>), ('candidate_movie_id', <tf.Tensor 'arg1:0' shape=(1,) dtype=int32>), ('rating', <tf.Tensor 'arg4:0' shape=(1,) dtype=int32>), ('query_movie_ids_len', <tf.Tensor 'ExpandDims:0' shape=(1,) dtype=int32>), ('genres_len', <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)])

features:  OrderedDict([('query_movie_ids', <tf.Tensor 'IteratorGetNext:4' shape=(?, ?) dtype=int32>), ('genres', <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int32>), ('avg_rating', <tf.Tensor 'IteratorGetNext:0' shape=(?, 1) dtype=float32>), ('year', <tf.Tensor 'IteratorGetNext:7' shape=(?, 1) dtype=float32>), ('candidate_movie_id', <tf.Tensor 'IteratorGetNext:1' shape=(?, 1) dtype=int32>), ('query_movie_ids_len', <tf.Tensor 'IteratorGetNext:5' shape=(

<br/>
<br/>
<br/>
## Local Transform

In [4]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

ctrl = reload('recomm.trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 'is_local': True}
ctrl.gen_data(params)

2018-03-13 09:49:14,173 - Loader - INFO [line:363] - try to unserialize from D:\Python\notebook\recomm_prod\repo/foo-bar/movielens_recommendation/data/parsed.yaml
2018-03-13 09:49:15,787 - Loader - INFO [line:381] - try to transform ['D:/Python/notebook/recomm_prod/data/foo/user_supplied/raws/merged_movielens.csv'] ... 


KeyboardInterrupt: 

<br/>
<br/>
<br/>
## Local View Schema

In [3]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

with flex.io('../repo/foo-bar/movielens_recommendation/data/parsed.yaml').as_reader() as f:
    schema = flex.Schema.unserialize(f.stream)
vars(schema)

{'col_states_': OrderedDict([('query_movie_ids',
               CatgMapper(allow_null=True, default=None, is_multi=True,
                     name='query_movie_ids', sep=',', vocabs=None, vocabs_path=None)),
              ('genres',
               CatgMapper(allow_null=True, default=None, is_multi=True, name='genres',
                     sep='|', vocabs=None, vocabs_path=None)),
              ('avg_rating', NumericMapper(default=None, name='avg_rating')),
              ('year', NumericMapper(default=None, name='year')),
              ('candidate_movie_id',
               CatgMapper(allow_null=True, default=None, is_multi=False,
                     name='candidate_movie_id', sep=None, vocabs=None, vocabs_path=None)),
              ('rating',
               CatgMapper(allow_null=False, default=None, is_multi=False, name='rating',
                     sep=None, vocabs=None, vocabs_path=None))]),
 'conf_': {'columns': [{'id': 'user_id', 'm_dtype': 'catg'},
   {'id': 'query_movie_ids',
  

<br/>
<br/>
<br/>
## Local Training

In [3]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 
          'is_local': True,
          'runtime_version': '1.4',
          'train_steps': 600}
result = views.train(params)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

## Local Get Model and Predict

In [231]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
est = reload('recomm.trainer.reco_mf_dnn_est')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 
          'is_local': True, 'json_data': restful_data((22,))}
r = views.est_predict(params)

INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B5095FE978>, '_service': None, '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': 88, '_log_step_count_steps': 300, '_evaluation_master': '', '_model_dir': 'D:\\Python\\notebook\\recomm_prod\\repo/foo-bar/movielens_recommendation/model', '_session_config': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100}
2018-03-09 15:54:09,202 - tensorflow - INFO [line:116] - Using config: {'_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B5095FE978>, '_service': None, '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_type': 'worker', '_ta

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [{'query_movie_ids_len': 153, 'query_movie_ids': [32, 1456, 1254, 1225, 1126, 1118, 1117, 1116, 1113, 1101, 1084, 1046, 1025, 1010, 995, 972, 967, 958, 1729, 1811, 1814, 2477, 3214, 3093, 3034, 2824, 2819, 2683, 2584, 2437, 1967, 2399, 2375, 2289, 2213, 2170, 2063, 2039, 956, 1404, 407, 562, 499, 428, 495, 523, 486, 525, 526, 226, 233, 143, 267, 696, 881, 2310, 208, 2402, 2373, 204, 2454, 185, 2397, 917, 2214, 2532, 240, 2181, 2175, 2174, 2169, 2162, 2161, 2148, 2140, 2478, 2646, 2554, 3001, 3217, 46, 3203, 3200, 3199, 3183, 47, 3060, 3046, 66, 2952, 153, 2894, 2861, 133, 138, 2744, 2720, 2682, 2095, 2637, 2610, 2130, 322, 2094, 522, 1346, 1337, 1310, 1306, 1295, 1280, 1235, 1130, 496, 520, 1115, 2085, 1112, 1066, 649, 1018, 650, 872, 971, 965, 959, 889, 1360, 1361, 1388, 43, 282, 321, 2005, 892, 1941, 1913, 1854, 1821, 392, 1765, 1676, 1642, 1586, 1574, 1569, 1560, 1487, 1481, 1474, 433, 1450, 3229], 'cand

## Test

In [245]:
utils = reload('recomm.trainer.utils.utils')
env = reload('recomm.trainer.env')
flex = reload('recomm.trainer.utils.flex')
est = reload('recomm.trainer.reco_mf_dnn_est')
service = reload('recomm.trainer.service')

views = reload('recomm.views').ViewRecomm.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
print(views.test(params).get('response'))

d:\python\anaconda3\envs\py3_5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['outputs']:
The given SavedModel SignatureDef contains the following input(s):
inputs['avg_rating'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1)
    name: avg_rating:0
inputs['candidate_movie_id'] tensor_info:
    dtype: DT_INT32
    shape: (-1)
    name: candidate_movie_id:0
inputs['genres'] tensor_info:
    dtype: DT_INT32
    shape: (-1, -1)
    name: genres:0
inputs['genres_len'] tensor_info:
    dtype: DT_INT32
    shape: (-1)
    name: genres_len:0
inputs['query_movie_ids'] tensor_info:
    dtype: DT_INT32
    shape: (-1, -1)
    name: query_movie_ids:0
inputs['query_movie_ids_len

### 更改GCS movielens.yaml

In [ ]:
import shutil
from google.cloud.storage.blob import Blob
from io import BytesIO

utils = reload('recomm.trainer.utils.utils')
flex = reload('recomm.trainer.utils.flex')
env = reload('recomm.trainer.env')

with flex.io('../data/foo/user_supplied/movielens.yaml') as r, \
    flex.io('gs://movielens-foo/user_supplied/movielens.yaml') as w:
    w.write(r.read())

# stream = BytesIO(open('../data/foo/user_supplied/movielens.yaml', mode='rb').read())
# utils.gcs_blob('gs://movielens-foo/user_supplied/movielens.yaml').upload_from_file(stream)

<br/>
<br/>
<br/>
## Dataset

In [ ]:
def make_datasets(fpath_ary, schema, n_batch=128, n_epoch=1):
    def to_dense(sp):
        dense = tf.sparse_to_dense(sp.indices, sp.dense_shape, sp.values, '')
        return tf.reshape(tf.to_int32(tf.string_to_number(dense)), [-1])

    def to_sparse(dense):
        idx = tf.where(tf.not_equal(dense, 0))
        return tf.SparseTensor(indices=idx, dense_shape=dense.get_shape(), values=tf.gather_nd(dense, idx))

    def parse_csv(value):
        data = tf.decode_csv(value, record_defaults=defaults)
        features = OrderedDict(zip(cols, data))
        multi_cols = df_conf.query("{} == '{}' and {} == True".format(schema.M_DTYPE, schema.CATG, schema.IS_MULTI)).id.values
        for col in multi_cols:
            features[col] = tf.string_split([features[col]], ',')
            features[col] = to_dense(features[col])
            # features['{}_lens'.format(col)] = tf.size(features[col])
        return features 
    
    df_conf = schema.df_conf_.query('{}.notnull()'.format(schema.TYPE))
    cols = schema.cols
    defaults = []
    for _, r in df_conf.iterrows():
        if r[schema.M_DTYPE] == schema.CATG:
            defaults.append([''] if r[schema.IS_MULTI] else [0])
        else:
            defaults.append([])
    dataset = tf.data.TextLineDataset(fpath_ary)
    dataset = dataset.map(parse_csv, num_parallel_calls=4)
    has_multi = (df_conf[schema.M_DTYPE] == schema.CATG) & (df_conf[schema.IS_MULTI] == True)
    if sum(has_multi):
        multi_cols = df_conf[has_multi].id.values
        dataset = dataset.padded_batch(n_batch, OrderedDict( zip(cols, tuple([None] if e else [] for e in has_multi))) )
    else:
        dataset = dataset.batch(n_batch)
    dataset = dataset.shuffle(n_batch * 10, seed=seed).repeat(n_epoch)
    features = dataset.make_one_shot_iterator().get_next()
    return features, features.pop(schema.label[0])
                                
# tf.reset_default_graph()
with tf.Graph().as_default():
    inputs = make_datasets(['./movielens.tr'], loader.schema, n_batch=30)
    query_lens = tf.sequence_mask([1, 2, 3])
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            _, = sess.run([inputs])
            # print( sess.run(inputs) )
            pass

## Feature Columns with tf.feature_column.input_layer

In [ ]:
a = pd.Series(minmax_scale(np.random.normal(0, 1, size=1000)))
a.hist(bins=50)

In [ ]:
%%time
tf.reset_default_graph()
with tf.Graph().as_default():
    user_id = tf.feature_column.categorical_column_with_hash_bucket('user_id', hash_bucket_size=1000, dtype=tf.int32)
    user_id = tf.feature_column.embedding_column(user_id, dimension=8)
    avg_rating = tf.feature_column.numeric_column('avg_rating')
    columns = [user_id, avg_rating]
    
    def make_datasets(fpath_ary):
        cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
        defaults = [[0], [''], [''], [], [], [0], []]

        def parse_csv(value):
            data = tf.decode_csv(value, record_defaults=defaults)
            features = OrderedDict(zip(cols, data))
            # print(features)
            return features
        
        dataset = tf.data.TextLineDataset(fpath_ary)
        dataset = (dataset.map(parse_csv, num_parallel_calls=4)
                          .batch(3)
                          # .padded_batch(3, OrderedDict(zip(cols, ([], [None], [None], [], [], [], []))))
                          .shuffle(10, seed=seed)
                          .repeat(1)
                  )
        return dataset.make_one_shot_iterator().get_next()
    
    inputs = make_datasets(['./te_processed.batch.csv'])
    inputs = tf.feature_column.input_layer(inputs, columns)
    # features = tf.parse_example(serialized_example, features=tf.feature_column.make_parse_example_spec(columns))
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            print(sess.run(inputs))

### Make Example

In [ ]:
%%time
cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
is_multi = [False, True, True, False, False, False, False]
pd_dtypes = [int, str, str, float, float, int, float]
types = ['int64_list', 'int64_list', 'int64_list', 'float_list', 'float_list', 'int64_list', 'float_list']
tf_types = [tf.int64, tf.int64, tf.int64, tf.float32, tf.float32, tf.int64, tf.float32]
def persist_example(fpath, tfpath):
    with tf.python_io.TFRecordWriter(tfpath) as w:
        for chunk in pd.read_csv(fpath, names=cols, dtype=dict(zip(cols, pd_dtypes)), chunksize=1000):
            chunk['query_movie_ids'] = chunk.query_movie_ids.map(lambda r: map(int, r.split(',')))
            chunk['genres'] = chunk.genres.map(lambda r: map(int, r.split(',')))
            
            for idx, r in chunk.iterrows():
                ex = tf.train.Example()
                for multi, col, tpe in zip(is_multi, cols, types):
                    val = r[col]
                    # ex.features.feature[col].int64_list or float_list or bytes_list
                    feat_type = getattr(ex.features.feature[col], tpe)
                    # extend function for multivalent columns, otherwise append
                    append_or_extend = 'append' if not multi else 'extend'                    
                    getattr(feat_type.value, append_or_extend)(val)
                w.write(ex.SerializePartialToString())

persist_example('./te_processed.csv', './data.tfrecord')

In [ ]:
def decode_example(ser_example):
    # queue = tf.train.string_input_producer([fpath], num_epochs=1)
    # _, ser_example = tf.TFRecordReader().read(queue)
    # ser_example = tf.train.batch([ser_example], batch_size=10)
    ctx_features = {col: tf.FixedLenFeature([], tf_tpe)
                    for col, tf_tpe in zip(cols, tf_types) if col not in ('query_movie_ids', 'genres')}
    seq_features = {col: tf.FixedLenSequenceFeature([], tf_tpe) 
                    for col, tf_tpe in [('query_movie_ids', tf.int64), ('genres', tf.int64)]}
    context_dict, sequence_dict = tf.parse_single_sequence_example(ser_example, 
                                                                   context_features=ctx_features, 
                                                                   sequence_features=seq_features)
    # for col, tpe in zip(cols, tf_types):
    #     val = feature_dict[col]
    #     feature_dict[col] = tf.sparse_to_dense(val.indices, val.dense_shape, val.values, name=col)
    feature_dict = {}
    feature_dict.update(context_dict)
    feature_dict.update(sequence_dict)
    ret = OrderedDict()
    for c in cols:
        ret[c] = feature_dict[c]
    return tuple(ret.values())

tf.reset_default_graph()
with tf.Graph().as_default():
    dataset = tf.data.TFRecordDataset(['./data.tfrecord'])
    dataset = dataset.map(decode_example).padded_batch(10, padded_shapes=([], [None], [None], [], [], [], []))
    # dataset = dataset.batch(3)
    iters = dataset.make_one_shot_iterator()
    r = iters.get_next()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        print( sess.run(r) )

## Traditional parse_example
1. tf.train.Coordinator + tf.train.start_queue_runners

In [ ]:
from tensorflow.python.framework import sparse_tensor
import re

def to_sparse(dense):
    idx = tf.where(tf.not_equal(dense, 0))
    return tf.SparseTensor(idx, tf.gather_nd(dense, idx), dense.get_shape())

def make_example(val):
    example = tf.train.Example(features=tf.train.Features(
        feature = {
            'query_movie_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=val)),
            'genres': tf.train.Feature(int64_list=tf.train.Int64List(value=val))
        }
    ))
    return example

tf.reset_default_graph()
with tf.Graph().as_default():
    
    filename = "tmp.tfrecords"
    if not os.path.exists(filename):
        # os.remove(filename)
        writer = tf.python_io.TFRecordWriter(filename)
        with writer:
            for idx, r in teProcessed.head().iterrows():
                for col in ('query_movie_ids', 'genres'):
                    val = list(map(int, re.split(',\s*', r[col])))
                    ex = make_example(val)
                    writer.write(ex.SerializeToString())

    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer(["tmp.tfrecords"], num_epochs=1)
    _, serialized_example = reader.read(filename_queue)

    batch = tf.train.batch(tensors=[serialized_example], batch_size=1)
    features = {
        'query_movie_ids': tf.VarLenFeature(tf.int64),
        'genres': tf.VarLenFeature(tf.int64)
    }
    data = tf.parse_example(batch, features)
    query_movie_ids = data['query_movie_ids']
    embbedding = tf.Variable(tf.glorot_uniform_initializer()([9125]), dtype=tf.float32)
    print(query_movie_ids.dense_shape)
    # r = tf.layers.dense(query_movie_ids, 10)
    # emb_query = tf.nn.embedding_lookup_sparse([embbedding], query_movie_ids, None, combiner='sqrtn')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        try:
            print(sess.run(data))
            pass
        except tf.errors.OutOfRangeError as e:
            coord.request_stop(e)
        finally:
            coord.request_stop()
            coord.join(threads)
    

## Test

In [205]:
tf.reset_default_graph()
with tf.Graph().as_default():
    # a = tf.constant([[0, 1], [0, 2]])
    # w = tf.constant([[1, 1, 1],
    #                  [2, 2, 2],
    #                  [3, 3, 3]])
    # w1 = tf.constant([[1, 1, 1],
    #                   [2, 2, 2],
    #                   [3, 3, 3]])
    # c = tf.nn.embedding_lookup(w, a)
    print(tf.constant([1, 2, 3]))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # print(tf.size(tf.constant([1, 2, 3])).eval())
        # print((w * w1).eval())
        # print(c.eval())
        # print(tf.sequence_mask(tf.constant([[1], [2], [3]])).eval())

Tensor("Const:0", shape=(3,), dtype=int32)
